# Dataset 확인

## 영상 길이 넘는지 확인

In [34]:
import cv2
import os
import pandas as pd

person_0 = []
person_1 = []
person_2 = []
person_3 = []
# person_4 = []

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
for person in data_per:
    words_dir= f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for id in Id:
        if id == '.DS_Store':
            continue
        id_dir = f'{data_dir}/{person}/{id}'
        num = os.listdir(id_dir)
        if len(num) == 3:
            num = os.listdir(id_dir)
            for n  in num:
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/image'
                frames = os.listdir(words_dir_2)
                if frames == '.DS_Store':
                    continue
                if len(frames) < 5:
                    if person == '00':
                        person_0.append((id, n))
                    elif person == '01':
                        person_1.append((id, n))
                    elif person == '02':
                        person_2.append((id, n))
#                     elif person == '03':
#                         person_3.append((id, n))
                    else:
                        person_1.append((id, n))
print(len(person_0),len(person_1))

0 0


## 데이터셋 전처리(crop)

* 입술 주변을 Detect후 100x50 pixel로 crop

In [35]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((20, 2), dtype=dtype)
    # for only lip landmarks
    n = 0
    for i in range(48, shape.num_parts):
        coords[n] = (shape.part(i).x, shape.part(i).y)
        n += 1

    # return the list of (x, y)-coordinates
    return coords

In [36]:
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat")

faces = f'korea_data/00/00/00/image'
for jpg in os.listdir(faces):
    if jpg[3:6] not in ['_0.', '_1.', '_2.', '_3.','_4']:
        pass
    else:
        dir = f'{faces}/{jpg}'
        src = cv2.imread(dir, cv2.IMREAD_COLOR)
        img_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        face_detector = detector(img_gray, 1)
        for face in face_detector:
            landmarks = predictor(src, face)
            landmarks = shape_to_np(landmarks)
            print("gotfaces!")

gotfaces!
gotfaces!
gotfaces!
gotfaces!


In [42]:
## Dataset crop Demo
from tensorflow.keras.preprocessing import image
import cv2
import os
import pandas as pd
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat") # predict 해주는애            

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
cnt = 0
for person in data_per:
    for sentence in os.listdir(f'{data_dir}/{person}' ):
        if sentence == 'csv' or sentence == 'video' or sentence == '.DS_Store':
            continue
        for nums in os.listdir(f'{data_dir}/{person}/{sentence}'):
            if nums == '.DS_Store':
                continue
            count = 0
            print(f'{data_dir}/{person}/{sentence}/{nums}/image')
            for jpg in os.listdir(f'{data_dir}/{person}/{sentence}/{nums}/image'):
                # 3초 안으로
                if jpg[3:6] not in ['_0.', '_1.', '_2.', '_3.', '_4.']:
                    pass
                else:
                    faces = f'{data_dir}/{person}/{sentence}/{nums}/image/{jpg}'
                    src = cv2.imread(faces, cv2.IMREAD_COLOR)
                    img_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
                    face_detector = detector(img_gray, 1)
                    for face in face_detector:
                        landmarks = predictor(src, face)
                        landmarks = shape_to_np(landmarks)
                        # select center of mouth
                        x_list = [x[0] for x in landmarks]
                        y_list = [y[1] for y in landmarks]
                        x = sum(x_list)//20
                        y = sum(y_list)//20
                    dst = src[y-50:y+50, x-100:x+100].copy()
                    # 확인용
#                     cv2.imshow("src", src)
#                     cv2.imshow("dst", dst)
#                     cv2.waitKey()
#                     cv2.destroyAllWindows()
                    img_tensor = image.img_to_array(dst) #tensor로 type 변경
                    img_tensor /= 255. # scaling  # (100, 200, 3)
                    img_tensor = np.expand_dims(img_tensor, axis = 0) #(1, 100, 200, 3)
                    # frame 별 묶기
                    if count == 0:
                        x_data = img_tensor
                        count += 1
                    else:
                        x_data = np.concatenate((x_data, img_tensor), axis = 0)
            # (1, 5, 100, 200, 3)
            x_data = np.expand_dims(x_data, axis = 0) 
            print("x_data:", x_data.shape)
            if cnt == 0:
                x_data_ = x_data
                cnt += 1
            else:
                x_data_ = np.concatenate((x_data_, x_data), axis = 0)
            print(x_data_.shape)
                #(갯수, frame, 64, 64, 3)

korea_data/00/00/00/image
x_data: (1, 5, 100, 200, 3)
(1, 5, 100, 200, 3)
korea_data/00/00/01/image
x_data: (1, 5, 100, 200, 3)
(2, 5, 100, 200, 3)
korea_data/00/00/02/image
x_data: (1, 5, 100, 200, 3)
(3, 5, 100, 200, 3)
korea_data/00/01/00/image
x_data: (1, 5, 100, 200, 3)
(4, 5, 100, 200, 3)
korea_data/00/01/01/image
x_data: (1, 5, 100, 200, 3)
(5, 5, 100, 200, 3)
korea_data/00/01/02/image
x_data: (1, 5, 100, 200, 3)
(6, 5, 100, 200, 3)
korea_data/00/02/00/image
x_data: (1, 5, 100, 200, 3)
(7, 5, 100, 200, 3)
korea_data/00/02/01/image
x_data: (1, 5, 100, 200, 3)
(8, 5, 100, 200, 3)
korea_data/00/02/02/image
x_data: (1, 5, 100, 200, 3)
(9, 5, 100, 200, 3)
korea_data/00/03/00/image
x_data: (1, 5, 100, 200, 3)
(10, 5, 100, 200, 3)
korea_data/00/03/01/image
x_data: (1, 5, 100, 200, 3)
(11, 5, 100, 200, 3)
korea_data/00/03/02/image
x_data: (1, 5, 100, 200, 3)
(12, 5, 100, 200, 3)
korea_data/00/04/00/image
x_data: (1, 5, 100, 200, 3)
(13, 5, 100, 200, 3)
korea_data/00/04/01/image
x_data: 

x_data: (1, 5, 100, 200, 3)
(110, 5, 100, 200, 3)
korea_data/00/36/02/image
x_data: (1, 5, 100, 200, 3)
(111, 5, 100, 200, 3)
korea_data/00/37/00/image
x_data: (1, 5, 100, 200, 3)
(112, 5, 100, 200, 3)
korea_data/00/37/01/image
x_data: (1, 5, 100, 200, 3)
(113, 5, 100, 200, 3)
korea_data/00/37/02/image
x_data: (1, 5, 100, 200, 3)
(114, 5, 100, 200, 3)
korea_data/00/38/00/image
x_data: (1, 5, 100, 200, 3)
(115, 5, 100, 200, 3)
korea_data/00/38/01/image
x_data: (1, 5, 100, 200, 3)
(116, 5, 100, 200, 3)
korea_data/00/38/02/image
x_data: (1, 5, 100, 200, 3)
(117, 5, 100, 200, 3)
korea_data/00/39/00/image
x_data: (1, 5, 100, 200, 3)
(118, 5, 100, 200, 3)
korea_data/00/39/01/image
x_data: (1, 5, 100, 200, 3)
(119, 5, 100, 200, 3)
korea_data/00/39/02/image
x_data: (1, 5, 100, 200, 3)
(120, 5, 100, 200, 3)
korea_data/00/40/00/image
x_data: (1, 5, 100, 200, 3)
(121, 5, 100, 200, 3)
korea_data/00/40/01/image
x_data: (1, 5, 100, 200, 3)
(122, 5, 100, 200, 3)
korea_data/00/40/02/image
x_data: (1, 

x_data: (1, 5, 100, 200, 3)
(218, 5, 100, 200, 3)
korea_data/01/27/02/image
x_data: (1, 5, 100, 200, 3)
(219, 5, 100, 200, 3)
korea_data/01/28/00/image
x_data: (1, 5, 100, 200, 3)
(220, 5, 100, 200, 3)
korea_data/01/28/01/image
x_data: (1, 5, 100, 200, 3)
(221, 5, 100, 200, 3)
korea_data/01/28/02/image
x_data: (1, 5, 100, 200, 3)
(222, 5, 100, 200, 3)
korea_data/01/29/00/image
x_data: (1, 5, 100, 200, 3)
(223, 5, 100, 200, 3)
korea_data/01/29/01/image
x_data: (1, 5, 100, 200, 3)
(224, 5, 100, 200, 3)
korea_data/01/29/02/image
x_data: (1, 5, 100, 200, 3)
(225, 5, 100, 200, 3)
korea_data/01/30/00/image
x_data: (1, 5, 100, 200, 3)
(226, 5, 100, 200, 3)
korea_data/01/30/01/image
x_data: (1, 5, 100, 200, 3)
(227, 5, 100, 200, 3)
korea_data/01/30/02/image
x_data: (1, 5, 100, 200, 3)
(228, 5, 100, 200, 3)
korea_data/01/31/00/image
x_data: (1, 5, 100, 200, 3)
(229, 5, 100, 200, 3)
korea_data/01/31/01/image
x_data: (1, 5, 100, 200, 3)
(230, 5, 100, 200, 3)
korea_data/01/31/02/image
x_data: (1, 

x_data: (1, 5, 100, 200, 3)
(326, 5, 100, 200, 3)
korea_data/02/18/02/image
x_data: (1, 5, 100, 200, 3)
(327, 5, 100, 200, 3)
korea_data/02/19/00/image
x_data: (1, 5, 100, 200, 3)
(328, 5, 100, 200, 3)
korea_data/02/19/01/image
x_data: (1, 5, 100, 200, 3)
(329, 5, 100, 200, 3)
korea_data/02/19/02/image
x_data: (1, 5, 100, 200, 3)
(330, 5, 100, 200, 3)
korea_data/02/20/00/image
x_data: (1, 5, 100, 200, 3)
(331, 5, 100, 200, 3)
korea_data/02/20/01/image
x_data: (1, 5, 100, 200, 3)
(332, 5, 100, 200, 3)
korea_data/02/20/02/image
x_data: (1, 5, 100, 200, 3)
(333, 5, 100, 200, 3)
korea_data/02/21/00/image
x_data: (1, 5, 100, 200, 3)
(334, 5, 100, 200, 3)
korea_data/02/21/01/image
x_data: (1, 5, 100, 200, 3)
(335, 5, 100, 200, 3)
korea_data/02/21/02/image
x_data: (1, 5, 100, 200, 3)
(336, 5, 100, 200, 3)
korea_data/02/22/00/image
x_data: (1, 5, 100, 200, 3)
(337, 5, 100, 200, 3)
korea_data/02/22/01/image
x_data: (1, 5, 100, 200, 3)
(338, 5, 100, 200, 3)
korea_data/02/22/02/image
x_data: (1, 

x_data: (1, 5, 100, 200, 3)
(434, 5, 100, 200, 3)
korea_data/03/09/02/image
x_data: (1, 5, 100, 200, 3)
(435, 5, 100, 200, 3)
korea_data/03/10/00/image
x_data: (1, 5, 100, 200, 3)
(436, 5, 100, 200, 3)
korea_data/03/10/01/image
x_data: (1, 5, 100, 200, 3)
(437, 5, 100, 200, 3)
korea_data/03/10/02/image
x_data: (1, 5, 100, 200, 3)
(438, 5, 100, 200, 3)
korea_data/03/11/00/image
x_data: (1, 5, 100, 200, 3)
(439, 5, 100, 200, 3)
korea_data/03/11/01/image
x_data: (1, 5, 100, 200, 3)
(440, 5, 100, 200, 3)
korea_data/03/11/02/image
x_data: (1, 5, 100, 200, 3)
(441, 5, 100, 200, 3)
korea_data/03/12/00/image
x_data: (1, 5, 100, 200, 3)
(442, 5, 100, 200, 3)
korea_data/03/12/01/image
x_data: (1, 5, 100, 200, 3)
(443, 5, 100, 200, 3)
korea_data/03/12/02/image
x_data: (1, 5, 100, 200, 3)
(444, 5, 100, 200, 3)
korea_data/03/13/00/image
x_data: (1, 5, 100, 200, 3)
(445, 5, 100, 200, 3)
korea_data/03/13/01/image
x_data: (1, 5, 100, 200, 3)
(446, 5, 100, 200, 3)
korea_data/03/13/02/image
x_data: (1, 

In [41]:
x_data.shape

(1, 5, 100, 200, 3)

In [12]:
np.save('x_data.npy', x_data_)

In [14]:
x_data = np.load('train/x_data.npy') 

In [15]:
x_data.shape

(540, 5, 100, 200, 3)

In [16]:
data = [0]*24 + [1]*3 + [2]*18 + [3]*6 + [4]*15 +[5]*12 + [6]*33 +[7]*3 + [8]*9 + [9]*3 + [10]*3 +[11]*3 + [12]*3

In [17]:
data = np.repeat(data, 4)

In [18]:
print(data.shape)
import keras
y_data = keras.utils.to_categorical(data, None) 
print(y_data.shape)

(540,)
(540, 13)


In [19]:
np.save('y_data.npy', y_data)

In [20]:
y_data = np.load('train/y_data.npy') 
y_data.shape

(540, 13)